# Read raw screen data from spreadsheets and reformat

In [1]:
import pandas
pandas.__version__

'1.0.3'

## Read FDA compounds

> The SCREENWELL FDA-approved drug library V2 containing 741 compounds was purchased from Enzo Life Sciences (Hayashi Kasei Co., Ltd.), and the International Drug Collection (IDC) containing 311 compounds was purchased from MicroSource Discovery Systems, Inc. (Namiki Shoji Co., Ltd.).

In [2]:
def undo_mangle_dupe_cols(df):
    """
    undo mangle_dupe_cols=True in pandas.read_excel
    Removes ".1" suffix.
    """
    strip_suffix = lambda x: x[:-2] if x.endswith(".1") else x
    df.columns = df.columns.map(strip_suffix)
    return df


def read_fda_spreadsheet(concentration):
    concentration_to_cols = {"1 uM": "B:H", "10 uM": "J:P"}
    renamer = {
        f"Period": "period_before",
        f"{concentration} AVG": "period_after",
        "Period Change": "period_change",
        "HITS/non-HITS": "compound_name",
        "Plate": "plate",
    }
    df = (
        pandas.read_excel(
            io="data/tamai-screen/FDA1_FDA2_Period_24-120.xlsx",
            sheet_name="FDA_1uM_10uM_DRUGS_GRAPHS",
            usecols=f"{concentration_to_cols[concentration]},T",
            mangle_dupe_cols=True,
        )
        .pipe(undo_mangle_dupe_cols)
        .rename(columns=renamer)
        .rename(columns=lambda x: "notes" if x.startswith("Unnamed:") else x)
        # .eval("dead = 0 + (notes == 'dead')")
        .assign(concentration = concentration)
        .dropna(subset=["period_after", "compound_name"])
        .assign(compound_set="fda")
        .convert_dtypes()
    )
    return df


In [3]:
fda_df = pandas.concat([
    read_fda_spreadsheet("1 uM"),
    read_fda_spreadsheet("10 uM"),
])
fda_df.head(2)

,96-well,plate,384-well,period_before,period_after,period_change,notes,compound_name,concentration,compound_set
0,1-A02,1,A4,21.5,21.633333,0.029167,<NA>,Clindamycin·HCl,1 uM,fda
3,1-A03,1,A7,21.7,21.666667,0.062500,<NA>,Felbamate,1 uM,fda


## Read IDC compounds

> the International Drug Collection (IDC) containing 311
compounds was purchased from MicroSource Discovery Systems, Inc.
(Namiki Shoji Co., Ltd.).

In [4]:
def read_idc_spreadsheet(concentration):
    concentration_to_cols = {"1 uM": "A:F", "10 uM": "G:L"}
    renamer = {
        "period": "period_before",
        "AVG": "period_after",
        f"IDC {concentration}": "period_change",
        "Name": "compound_name",
    }
    df = (
        pandas.read_excel(
            io="data/tamai-screen/IDC_Period_24-120.xlsx",
            sheet_name="1uM_10uM_Period_Graphs",
            usecols=f"{concentration_to_cols[concentration]},P",
            mangle_dupe_cols=True,
        )
        .pipe(undo_mangle_dupe_cols)
        .rename(columns=renamer)
        .rename(columns=lambda x: "notes" if x.startswith("Unnamed:") else x)
        .assign(concentration = concentration)
        .dropna(subset=["period_after"])
        .assign(compound_set="idc")
        .convert_dtypes()
    )
    return df

In [5]:
idc_df = pandas.concat([
    read_idc_spreadsheet("1 uM"),
    read_idc_spreadsheet("10 uM"),
])
idc_df.head(2)

,plate,well,period_before,period_after,period_change,notes,compound_name,concentration,compound_set
0,1,A4,21.9,22.0,0.01875,<NA>,SULFACARBAMIDE,1 uM,idc
3,1,A7,21.7,21.9,-0.08125,<NA>,GALLIC ACID,1 uM,idc


## Combine compound sets

In [6]:
screen_df = (
    pandas.concat([fda_df, idc_df])
    .convert_dtypes()
    .reindex(columns=[
        "compound_name",
        "compound_set",
        "concentration",
        "period_before",
        "period_change",
        "plate",
        "notes",
    ])
)
screen_df

,compound_name,compound_set,concentration,period_before,period_change,plate,notes
0,Clindamycin·HCl,fda,1 uM,21.5,0.029167,1,<NA>
3,Felbamate,fda,1 uM,21.7,0.062500,1,<NA>
6,Cyclosporine A,fda,1 uM,21.6,0.095833,1,<NA>
9,Donepezil·HCl,fda,1 uM,21.4,-0.170833,1,<NA>
12,Lincomycin·HCl,fda,1 uM,21.5,-0.004167,1,<NA>
...,...,...,...,...,...,...,...
945,METICRANE,idc,10 uM,21.5,-0.183333,6,<NA>
948,MEPARFYLON,idc,10 uM,21.4,-0.116667,6,<NA>
951,PICONOL,idc,10 uM,21.6,0.016667,6,<NA>
954,MENBUTONE,idc,10 uM,21.6,0.016667,6,<NA>


In [7]:
screen_df.to_csv("data/tamai-screen/combined-results.tsv", sep="\t", index=False, float_format="%.4g")